- Họ và tên: Phạm Trần Yến Quyên
- MSSV: 22127357
- Lớp: 22CLC02

# PROJECT 3 - Phân rã QR bằng Gram-Schmidt.
- Input: Ma trận $A$ là ma trận vuông thực $R^{n}$.
- Output: Trả về $A = QR$. Trong đó:
    + Q: ma trận trực chuẩn.
    + R: ma trận tam giác trên.
- **HOẶC** thông báo không thể phân rã QR

## TEST CASES

In [257]:
bt1 = [
    [1,1,2],
    [2,-1,1],
    [-2,4,1]
]

bt2 = [
    [1,1,1],
    [2,-2,2],
    [1,1,-1]
]

bt3 = [
    [1,1,-1],
    [0,1,2],
    [1,1,1]
]

bt4 = [
    [-1,-1,1],
    [1,3,3],
    [-1,-1,5],
    [1,3,7]
]

bt5 = [
    [1,1,1],
    [2,2,0],
    [-3,0,0],
    [0,0,1]
]

bt6 = [
    [-2,1,3],
    [1,0,0],
    [0,1,0],
    [0,0,1]
]

bt7 = [
    [1,-1,2],
    [1,0,-1],
    [-1,1,2],
    [0,1,1]
]

bt = [bt1, bt2, bt3, bt4, bt5, bt6, bt7]

# `QR_decomp()`

In [258]:
from fractions import Fraction
import time

# Sử dụng OOP làm class vector hỗ trợ các phép toán trên vector
class vector:
    def __init__(self, v):
        self.v = v
        self.n = len(v)    
        
    def __len__(self):
        return self.n
    
    def __sub__(self, v2):
        if self.n != v2.n:
            return "Không thể trừ 2 vecto có số chiều khác nhau"
        return vector([self.v[i] - v2.v[i] for i in range(self.n)])
    
    def __mul__(self, num):
        return vector([self.v[i]*num for i in range(self.n)])

    def __truediv__(self, num):
        return vector([self.v[i]/num for i in range(self.n)])
    
    def __getitem__(self, index):
        return self.v[index]
    
    def __setitem__(self, index, value):
        self.v[index] = value
    
    def if_all_zero(self):
        return all([self.v[i] == 0 or abs(self.v[i]) < 1e-10 for i in range(len(self.v))])    
        
    def dot_product(self, v2):
        if self.n != v2.n:
            return "Không thể nhân 2 vecto có số chiều khác nhau"
        return sum([self.v[i]*v2.v[i] for i in range(len(self.v))]) 
    
    def norm(self):
        norm_value = sum([self.v[i]**2 for i in range(len(self.v))])**0.5 # norm = sqrt(sum(v_i^2))
        if abs(norm_value) < 1e-10: # Làm tròn số nhỏ hơn 1e-10 về 0 (Floating point error)
            return 0
        return norm_value

    
    def print_vector(self):
        for i in range(self.n):
            if abs(self.v[i]) < 1e-10: # Làm tròn số nhỏ hơn 1e-10 về 0 (Floating point error)
                self.v[i] = 0
        print(self.v)

class Matrix:
    def __init__(self, A):
        self.A = A
        self.n = len(A) # Số hàng
        self.m = len(A[0]) # Số cột
         
    def if_square(self):
        # Kiểm tra ma trận vuông
        if self.n == self.m:
            return 1
        return 0
    
    def transpose(self):
        # Chuyển vị ma trận
        return Matrix([[self.A[j][i] for j in range(self.n)] for i in range(self.m)])
    
    def get_i_col(self, i):
        return [self.A[j][i] for j in range(self.n)]
            
    def get_linear_indp_set(self):
        # Tìm tập hợp tuyến tính độc lập
        # Lấy từng cột của ma trận, nếu cột đó không phải là tổ hợp tuyến tính của các cột trước nó thì thêm vào tập hợp        
        u_set = []
        for i in range(self.m):
            u = vector(self.get_i_col(i))
            if not any([u.dot_product(u_set[j]) == u.norm()*u_set[j].norm() for j in range(len(u_set))]): # Kiểm tra u có phải là tổ hợp tuyến tính của các u trước không
                u_set.append(u)
        return u_set
        
    def Gram_Schmidt(self, u_set): 
        # Tính tập hợp vecto chuẩn hóa bằng Gram-Schmidt
        v_set = []
        for i in range(len(u_set)):
            u = u_set[i]
            for j in range(i):
                u = u - v_set[j] * (u_set[i].dot_product(v_set[j]) / v_set[j].dot_product(v_set[j])) # u = u - (u.v_j)/(v_j.v_j)*v_j
            v_set.append(u / u.norm()) # v = u/||u||
        return v_set
        
    def print_matrix(self):
        for i in range(self.n):
            for j in range(self.m):
                if abs(self.A[i][j]) < 1e-10:
                    self.A[i][j] = 0
        for i in range(self.n):
            print(self.A[i])
        
    def QR_decomp(self):
        # Bước 1: Tìm tập hợp tuyến tính độc lập
        u_set = self.get_linear_indp_set() 
        """ print("Tập hợp tuyến tính độc lập:")
        for i in range(len(u_set)):
            u_set[i].print_vector() """

        # Bước 2: Tính tập hợp vecto chuẩn hóa bằng cách chia cho norm của vecto u
        v_set = self.Gram_Schmidt(u_set)
        """ print("Tập hợp vecto chuẩn hóa:")
        for i in range(len(v_set)):
            v_set[i].print_vector() """
            
        # Bước 3: Tính ma trận Q
        Q = Matrix([v_set[i].v for i in range(len(v_set))]).transpose() # Q = v_set^T
        
        # Bước 4: Tính ma trận R
        R = Matrix([[0 for i in range(self.m)] for j in range(self.m)]) # Khởi tạo ma trận R (mxm)
        for i in range(self.m):
            for j in range(i, self.m):
                R.A[i][j] = sum([self.A[k][j]*v_set[i].v[k] for k in range(self.n)]) # R = Q^T*A
        
        return Q, R
        
    

## MỞ RỘNG
- Tìm hiểu hàm/ phương thức tương ứng của các thư viện và thực hiện nó, so sánh kết quả.

In [259]:
import numpy as np
import sympy as sp

def solveWithLib(A):
    # numpy  
    time1 = time.time()
    print("\nNumpy QR Decomposition:")
    Q, R = np.linalg.qr(A)
    print("Thời gian thực thi Numpy: ", time.time() - time1)
    print("Q:")
    print(Q)
    print("R:")
    print(R)
    # sympy
    print("\nSympy QR Decomposition:")
    A = sp.Matrix(A)
    time2 = time.time()
    Q, R = A.QRdecomposition()
    print("Thời gian thực thi Sympy: ", time.time() - time2)
    print("Q:")
    print(Q)
    print("R:")
    print(R)
    return
    

# MAIN 
## Input: Ma trận vuông $A$
## Output: Phân rã QR của ma trận $A$ (nếu có).

In [260]:
def main():    
    # Hàm tự viết
    for i in range(len(bt)):
        A = Matrix(bt[i])
        time1 = time.time()
        Q, R = A.QR_decomp()
        print("\nThời gian thực hiện phân rã QR cho bài tập", i+1, "là:", time.time() - time1)
        print("Ma trận A sau khi phân rã QR có: ")
        print("Q = ")
        Q.print_matrix()
        print("R = ")
        R.print_matrix()
    print("-------------------------------------------------")
            
    # Hàm có sẵn
    for i in range(len(bt)):
        solveWithLib(bt[i])
    print("-------------------------------------------------")
    

if __name__ == "__main__":
    main()


Thời gian thực hiện phân rã QR cho bài tập 1 là: 0.0
Ma trận A sau khi phân rã QR có: 
Q = 
[0.3333333333333333, 0.6666666666666666, 0.6666666666666671]
[0.6666666666666666, 0.3333333333333333, -0.6666666666666664]
[-0.6666666666666666, 0.6666666666666666, -0.33333333333333287]
R = 
[3.0, -3.0, 0.6666666666666666]
[0, 3.0, 2.333333333333333]
[0, 0, 0.33333333333333487]

Thời gian thực hiện phân rã QR cho bài tập 2 là: 0.0
Ma trận A sau khi phân rã QR có: 
Q = 
[0.4082482904638631, 0.5773502691896256, 0.7071067811865476]
[0.8164965809277261, -0.5773502691896257, 0]
[0.4082482904638631, 0.5773502691896256, -0.7071067811865475]
R = 
[2.4494897427831783, -0.8164965809277261, 1.6329931618554523]
[0, 2.309401076758503, -1.1547005383792515]
[0, 0, 1.414213562373095]

Thời gian thực hiện phân rã QR cho bài tập 3 là: 0.0
Ma trận A sau khi phân rã QR có: 
Q = 
[0.7071067811865475, 0, -0.7071067811865475]
[0, 1.0, 0]
[0.7071067811865475, 0, 0.7071067811865475]
R = 
[1.414213562373095, 1.41421356

# Nhận xét chung:

## Thuật toán Gram-Schmidt:
### Ý tưởng chính:
1. Tìm tập hợp trực chuẩn:
    - Bước 1: Từ ma trận $A$, lập các vecto vô hướng ($u_1 → u_n$) từ các cột của $A$.
    - Bước 2: Tìm dãy vecto trực giao thông qua trực giao hóa Gram-Schmidt.
        + S1: Đặt $v_1 = u_1$.
        + S2: Đặt $v_2 = u_2 - \frac{<u_2, v_1>}{||v_1||^2}v_1$.
        + S3: Đặt $v_3 = u_3 - \frac{<u_3, v_1>}{||v_1||^2}v_1 - \frac{<u_3, v_2>}{||v_2||^2}v_2$.
        + .......
        > $v_i = u_i - \sum_{i=1}^n \frac{<u_i, v_j>}{||v_j||^2}v_j$ $(i ∈ [1,n])$
        > Nếu có trường hợp $v_i = 0$: Thông báo không độc lập tuyến tính và kết thúc.
    - Bước 3: Tìm vecto chuẩn hóa thông qua trực chuẩn hóa Gram-Schmidt. Bằng cách lấy các vecto trực giao chia cho độ dài chính nó.
        > $q_i = \frac{v_i}{||v_i||}$


2. Lập ma trận trực chuẩn $Q$ từ các vecto chuẩn hóa. <br>
$$Q = (q_1, q_2,...,q_n)$$

3. Lập ma trận tam giác trên $R= $.
\begin{pmatrix}
  <u_1, q_1>       & <u_2, q_1>   & \cdots  & <u_n, q_1>  \\
  <u_1, q_2>       & <u_2, q_2>   & \cdots  & <u_n, q_2>  \\
  \vdots           & \vdots       & \vdots  & \vdots \\
  <u_1, q_n>       & <u_2, q_n>   & \cdots  & <u_n, q_n>  \\
\end{pmatrix}

## Tự làm (Manual):

### Ý tưởng:
- Bao gồm hai class chính: `vector` và `Matrix`: 
  + `Class vector`: xây dựng dưới phương thức OOP hỗ trợ các phép toán cơ bản trên vector như trừ, nhân, chia, và các phép cụ thể để tính giá trị hỗ trợ cho quá trình phân rã: `norm(), dot__product()` của các vector. 
  + `Class Matrix`: hỗ trợ các phép toán trên ma trận, bao gồm việc tìm tập hợp các vector tuyến tính độc lập, chuẩn hóa các vector bằng phương pháp Gram-Schmidt, và phân tích QR của ma trận.

## Mô tả các hàm:

### Lớp `vector`

#### `__init__(self, v)`
- **Chức năng:** Khởi tạo một đối tượng vector.
- **Tham số:**
  - `v`: Danh sách các phần tử của vector.

#### `__len__(self)`
- **Chức năng:** Trả về số chiều của vector.
- **Trả về:** Số chiều của vector.

#### `__sub__(self, v2)`
- **Chức năng:** Trừ hai vector.
- **Tham số:**
  - `v2`: Đối tượng vector thứ hai.
- **Trả về:** Một vector mới là hiệu của hai vector nếu chúng có cùng số chiều, ngược lại trả về thông báo lỗi.

#### `__mul__(self, num)`
- **Chức năng:** Nhân vector với một số.
- **Tham số:**
  - `num`: Số để nhân với vector.
- **Trả về:** Một vector mới là kết quả của phép nhân.

#### `__truediv__(self, num)`
- **Chức năng:** Chia vector cho một số.
- **Tham số:**
  - `num`: Số để chia vector.
- **Trả về:** Một vector mới là kết quả của phép chia.

#### `__getitem__(self, index)`
- **Chức năng:** Lấy giá trị phần tử tại vị trí `index` trong vector.
- **Tham số:**
  - `index`: Vị trí của phần tử.
- **Trả về:** Giá trị của phần tử tại vị trí `index`.

#### `__setitem__(self, index, value)`
- **Chức năng:** Đặt giá trị phần tử tại vị trí `index` trong vector.
- **Tham số:**
  - `index`: Vị trí của phần tử.
  - `value`: Giá trị mới của phần tử.

#### `if_all_zero(self)`
- **Chức năng:** Kiểm tra xem tất cả các phần tử của vector có bằng 0 hay không.
- **Trả về:** `True` nếu tất cả các phần tử bằng 0, ngược lại `False`.

#### `dot_product(self, v2)`
- **Chức năng:** Tính tích vô hướng của hai vector.
- **Tham số:**
  - `v2`: Đối tượng vector thứ hai.
- **Trả về:** Giá trị của tích vô hướng nếu hai vector có cùng số chiều, ngược lại trả về thông báo lỗi.

#### `norm(self)`
- **Chức năng:** Tính norm của vector.
- **Trả về:** Giá trị norm của vector.

#### `print_vector(self)`
- **Chức năng:** In ra các phần tử của vector.

### Lớp `Matrix`

#### `__init__(self, A)`
- **Chức năng:** Khởi tạo một đối tượng ma trận.
- **Tham số:**
  - `A`: Danh sách hai chiều biểu diễn ma trận.

#### `if_square(self)`
- **Chức năng:** Kiểm tra xem ma trận có phải là ma trận vuông hay không.
- **Trả về:** `1` nếu ma trận vuông, ngược lại `0`.

#### `transpose(self)`
- **Chức năng:** Chuyển vị của ma trận.
- **Trả về:** Ma trận chuyển vị của ma trận hiện tại.

#### `get_i_col(self, i)`
- **Chức năng:** Lấy cột thứ `i` của ma trận.
- **Tham số:**
  - `i`: Vị trí của cột cần lấy.
- **Trả về:** Danh sách các phần tử trong cột thứ `i`.

#### `get_linear_indp_set(self)`
- **Chức năng:** Tìm tập hợp các vector tuyến tính độc lập từ các cột của ma trận.
- **Trả về:** Danh sách các vector tuyến tính độc lập.

#### `Gram_Schmidt(self, u_set)`
- **Chức năng:** Tính tập hợp các vector chuẩn hóa bằng phương pháp Gram-Schmidt.
- **Tham số:**
  - `u_set`: Danh sách các vector tuyến tính độc lập.
- **Trả về:** Danh sách các vector chuẩn hóa.

#### `print_matrix(self)`
- **Chức năng:** In ra các phần tử của ma trận.

#### `QR_decomp(self)`
- **Chức năng:** Dựa trên trình bày lý thuyết ở mục đầu tiên, phân tích QR của ma trận.
- **Trả về:** Cặp ma trận `Q` và `R` là kết quả của phân tích QR.

## Các bước chính của phân rã QR bằng Gram-Schmidt:
### Tìm tập hợp các vector tuyến tính độc lập:
- Khởi tạo các vector `u` là cột của ma trận ban đầu và tổng hợp trong `u_set`.
- **Kiểm tra tính độc lập tuyến tính**: Dòng `u.dot_product(u_set[j]) == u.norm() * u_set[j].norm()` so sánh điều kiện để xác định liệu vector `u` có phải là tổ hợp tuyến tính của `u_set[j]` hay không. Nếu điều kiện này không thoả mãn với bất kỳ `j` nào trong `u_set`, nghĩa là `u` là vector độc lập tuyến tính so với các vector trong `u_set`.


### Tìm vector chuẩn hóa:
1. **Khởi tạo tập hợp vector chuẩn hóa `v_set` rỗng.**
2. **Với mỗi vector `u` trong tập hợp `u_set` (tập hợp các vector tuyến tính độc lập):**
   - Tính `u` mới bằng cách trừ đi các thành phần của các vector đã chuẩn hóa trong `v_set`.
   - Chuẩn hóa `u` bằng cách chia cho norm của nó.
   - Thêm vector chuẩn hóa vào `v_set`.
3. **Trả về `v_set` là tập hợp các vector chuẩn hóa.**

### Tạo ma trận Q:
- Sau khi có được tập hợp các vector chuẩn hóa `v_set`, xếp các vector này **lần lượt là CỘT** để tạo thành ma trận `Q`.

### Tính ma trận R:
1. **Khởi tạo ma trận `R` rỗng có kích thước mxm (với m là số cột của ma trận ban đầu).**
2. **Với mỗi cặp chỉ số `(i, j)` trong ma trận `R`:**
   - Tính phần tử `R[i][j]` bằng tích vô hướng của cột thứ `j` của ma trận ban đầu và vector chuẩn hóa thứ `i`.
   - Lặp qua các hàng `k` của ma trận ban đầu để tính tổng các tích vô hướng.
3. **Trả về cặp ma trận `Q` và `R`.**

## Mở rộng:
### Ứng dụng của Phân rã QR:
#### 1. Least Squares problem (LS Problem):
- Least Squares problem thường xuất hiện khi cần điều chỉnh mô hình hoặc ước tính các tham số trong mô hình dựa trên dữ liệu thực tế. Việc sử dụng phân rã QR trong LS sẽ giúp giảm bớt phép tính và giải quyết nó hiệu quả hơn. Qua đó, ta có thể ước tính các tham số mô hình một cách chính xác và nhanh chóng từ dữ liệu , đặc biệt là khi $A$ là một ma trận vuông hoặc có hạng khớp (full rank).

##### **Giải thích tổng quát** 
- LS Problem có mục tiêu là tìm một vector $x$ sao cho $|Ax - b|_2^2$ là nhỏ nhất, trong đó:
  + $A$ là một ma trận $m*n$ với $m \geq n$.
  + $x$ là vector cần tìm. 
  + $b$ là vector dữ liệu.
##### **Phân rã QR của ma trận $A$** 
- Trước tiên, ma trận $A$ được phân rã thành tích của một ma trận $Q$ và một ma trận $R (A = QR)$:
  + $Q$ là ma trận trực chuẩn (nghĩa là $Q^T Q = I$).
  + $R$ là ma trận tam giác trên. <br>
##### **Thay thế vào bài toán** 
- Thay $A$ bằng $QR$, khi đó: $|Ax - b\|_2^2 = \|QRx - b\|_2^2 = \|Q(Rx - Q^T b)|_2^2$.
##### **Giải bài toán thay thế** 
- Bài toán mới sau khi thay thế trở thành tìm $x$ sao cho $|Rx - Q^T b\|_2^2$ là nhỏ nhất. Vì $Q$ là ma trận trực chuẩn như đã nói trên nên tính $Q^T b$ sẽ đơn giản và hiệu quả.
##### **Giải hệ phương trình tam giác trên** 
- Sau khi có $R$ và $Q^T b$, tìm $x$ bằng cách giải hệ phương trình tam giác trên $Rx = Q^T b$.

#### 2. Kalman Filter và ứng dụng của nó trong Vehicle Model
##### **Mục đích của Kalman Filter**
- Kalman Filter là một công cụ xử lý dữ liệu thống kê, được thiết kế để ước tính và dự đoán trạng thái của một hệ thống dựa trên dữ liệu đo được và mô hình của hệ thống đó. Mục đích chính của Kalman Filter là cải thiện độ chính xác của ước tính bằng cách kết hợp thông tin từ mô hình hệ thống và các đo lường thực tế, đồng thời xử lý nhiễu và sai số có thể xuất hiện trong quá trình đo lường.
##### **Ứng dụng của phân rã QR trong Kalman Filter**
- Trong Kalman Filter, phân rã QR được sử dụng để giải quyết vấn đề của ma trận không xác định, khi ma trận có định thức gần bằng 0 hoặc gần không khả nghịch. Điều này thường xảy ra trong Kalman Filter khi ma trận hiệp phương sai (covariance matrix) của hệ thống trở nên không ổn định do dữ liệu đo lường không chính xác hoặc không đủ.
- Khi sử dụng phân rã QR trong Kalman Filter, mục đích chính là cải thiện tính ổn định và chính xác của quá trình ước tính. Thay vì giải trực tiếp các phương trình đại số tuyến tính, phân rã QR cho phép tái cấu trúc ma trận một cách hiệu quả hơn bằng cách chia thành các thành phần trực giao và một ma trận tam giác trên. Việc này giúp giảm thiểu sai số và đảm bảo tính ổn định của Kalman Filter trong quá trình hoạt động.
##### **Extended Kalman Filter (EKF)**
- Ra đời với mục đích giải quyết vấn đề của Kalman Filter khi áp dụng cho các **hệ thống phi tuyến tính**. Vì về căn bản, Kalman Filter cơ bản làm việc với hệ thống tuyến tính. Điều này có nghĩa là các ma trận và phương trình mô tả phải là tuyến tính, nếu không thì Kalman Filter sẽ không thể áp dụng trực tiếp.
- **Mô hình hệ thống phi tuyến tính**: 
  + Là một mô hình toán học của một hệ thống dựa trên việc sử dụng một toán tử tuyến tính. Trong toán học, một phép toán tử tuyến tính (biến đổi tuyến tính/ánh xạ tuyến tính - linear transformation/linear mapping) là một ánh xạ $V → W$ giữa hai mô đun (cụ thể, hai không gian vector) mà bảo toàn được các thao tác cộng và nhân vô hướng vector.
  + Trong nhiều ứng dụng thực tế như điều khiển robot, điều khiển tự động, hoặc các hệ thống định vị và định hướng, các mô hình toán học thường có tính chất phi tuyến tính. Các hệ thống này có thể có các hàm phi tuyến tính, các biến số trạng thái không đồng nhất, hoặc các ma trận hiệp phương sai không còn tuyến tính.

- Extended Kalman Filter (EKF) làm việc với hệ thống phi tuyến tính: EKF là một phương pháp mở rộng của Kalman Filter, cho phép áp dụng cho các hệ thống phi tuyến tính bằng cách sử dụng các phép đạo hàm xấp xỉ (linearization). Cụ thể, EKF sử dụng các đạo hàm của các hàm phi tuyến tính xung quanh các điểm ước tính hiện tại của trạng thái hệ thống để xây dựng ma trận Jacobian và các phương trình tuyến tính hóa, thay vì sử dụng trực tiếp các biến số tuyến tính như Kalman Filter.
- **Linearization và tính ổn định**: Mặc dù EKF giải quyết được vấn đề cho các hệ thống phi tuyến tính, việc linearization (tuyến tính hóa) có thể dẫn đến sai số trong các ước tính. Điều này làm giảm tính chính xác của EKF so với KF trong các hệ thống có sự biến động lớn.

##### **Kalman Filter trong Vehicle Model**
Ứng dụng của Kalman Filter trong Vehicle Model được sử dụng để giải quyết nhiều vấn đề trong điều khiển và theo dõi phương tiện. Dưới đây là một số ứng dụng chính của Kalman Filter trong Vehicle Model:
- Điều khiển phương tiện tự lái (Autonomous Driving):
  + Kalman Filter có thể được sử dụng để ước tính và dự đoán vị trí, vận tốc, hướng di chuyển của phương tiện dựa trên các dữ liệu từ cảm biến như GPS, IMU (Inertial Measurement Unit), radar, camera, và lidar.
  + Bằng cách kết hợp các đo đạc từ các cảm biến này với mô hình phương tiện và các giả định về động lực học của xe, Kalman Filter giúp cải thiện độ chính xác của việc dự đoán và điều khiển phương tiện tự lái.
- Điều khiển động cơ và hộp số (Engine and Transmission Control):
  + Kalman Filter có thể ứng dụng để theo dõi và điều khiển các thông số của động cơ và hộp số như tốc độ quay động cơ, mô-men xoắn, v.v. Qua việc ước tính các thông số này, Kalman Filter giúp tối ưu hóa hiệu suất vận hành của phương tiện, từ đó cải thiện tiêu thụ nhiên liệu và hiệu quả của hệ thống.
- v.v.

##### **Ví dụ cụ thể**
- Để minh họa cách Kalman Filter có thể được áp dụng trong Vehicle Model. Dưới đây là mô hình của một hệ thống theo dõi vị trí và tốc độ của một phương tiện di chuyển trên một đoạn đường.

- Giả sử chúng ta đang sử dụng Kalman Filter để theo dõi phương tiện trên một đoạn đường. Các thông số có thể như sau:
  + $p(0) = 0$ (vị trí ban đầu của phương tiện).
  + $v(0) = 10$ (vận tốc ban đầu của phương tiện).
  + $\Delta t = 1$ (khoảng thời gian giữa các lần đo đạc).
  + $a(k)$ (gia tốc của phương tiện, có thể giả định hoặc đo từ cảm biến).
  + $\text{meas}_p(k)$ và $\text{meas}_v(k)$ (được đo từ GPS hoặc cảm biến vị trí và vận tốc).

1. Mô hình của phương tiện
- **Trạng thái hệ thống**: Có thể xác định trạng thái của phương tiện bằng vector $\mathbf{x} = [p, v]^T$, trong đó:
  + $p$: Vị trí của phương tiện trên đoạn đường (biểu diễn dưới dạng trục).
  + $v$: Vận tốc của phương tiện.

- **Mô hình diễn giải trạng thái**: Các phương trình sau có thể được sử dụng để mô tả sự biến đổi của trạng thái phương tiện:
$$ p(k+1) = p(k) + v(k) \Delta t $$
$$ v(k+1) = v(k) + a(k) \Delta t $$
- Trong đó:
  + $\Delta t$: Khoảng thời gian giữa các lần đo đạc.
  + $a(k)$: Gia tốc của phương tiện tại thời điểm $k$.

2. Quá trình đo lường 
- **Đo lường trực tiếp**: Sử dụng các cảm biến hoặc GPS để đo lường trực tiếp vị trí $\text{meas}_p(k)$ và vận tốc $\text{meas}_v(k)$ của phương tiện.

3. Công thức Kalman Filter 
- **Dự đoán (Prediction):**
$$ \hat{\mathbf{x}}^- (k+1) = A \hat{\mathbf{x}}(k) + B u(k) $$
$$ P^- (k+1) = A P(k) A^T + Q $$

- **Cập nhật (Update):**
$$ K(k+1) = P^- (k+1) H^T (H P^- (k+1) H^T + R)^{-1} $$
$$ \hat{\mathbf{x}}(k+1) = \hat{\mathbf{x}}^- (k+1) + K(k+1)[\mathbf{z}(k+1) - H \hat{\mathbf{x}}^- (k+1)] $$
$$ P(k+1) = (I - K(k+1) H) P^- (k+1) $$

$=>$ Từ đó, chúng ta có thể áp dụng Kalman Filter để ước tính và dự đoán vị trí và vận tốc của phương tiện dựa trên các đo lường thực tế và mô hình dự đoán. Quá trình này giúp cải thiện độ chính xác và độ tin cậy của các ước tính trong thời gian thực của hệ thống theo dõi phương tiện.

##### **Nguồn**: 
- 1. QR decomposition: History and its Applications - Tin-Yau Tam. (https://people.duke.edu/~hpgavin/SystemID/References/Tam-QR-history-2010.pdf) 
- 2. A study of QR decomposition and Kalman filter implementations - DAVID FUERTES RONCERO. (http://kth.diva-portal.org/smash/get/diva2:808731/FULLTEXT01.pdf)


### Tìm hiểu thư viện:

#### Thư viện Numpy
- Thời gian thực thi rất nhanh và hiệu quả. Lý do là vì NumPy được viết bằng C và Fortran (tham khảo nguồn qua github), nên thường nhanh hơn so với SymPy trong các tính toán lớn.
- NumPy có thể không đảm bảo chính xác toán học cao như SymPy, đặc biệt là trong các tính toán với các biểu thức phức tạp ( vẫn còn hiển thị kết quả có kí hiệu `$e-$` của số Floating point cực nhỏ).
- Có hỗ trợ phân rã QR cho cả ma trận số thực và số phức.

#### Thư viện Sympy
- Thời gian thực thi phân rã của Sympy chậm hơn so với Manual và Numpy nhưng vẫn tương đối hiệu quả. Nguyên nhân có thể là do thời gian chuyển kết quả sang dạng Fraction.
- Đảm bảo tính chính xác toán học cao (có thể xử lý biểu thức ký hiệu và biến số, phù hợp cho các bài toán yêu cầu chính xác toán học cao), giúp tránh lỗi làm tròn và sai số do tính số học hữu hạn.
